In [1]:
import tarfile
import urllib
import pandas as pd
import numpy as np

# Reads in the csv-files and creates a dataframe using pandas
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')


In [2]:
from zlib import crc32
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [3]:
garbage, train_set = split_train_test_by_id(train, 0.1, "Id")

In [4]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147 entries, 22 to 1452
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             147 non-null    int64  
 1   MSSubClass     147 non-null    int64  
 2   MSZoning       147 non-null    object 
 3   LotFrontage    122 non-null    float64
 4   LotArea        147 non-null    int64  
 5   Street         147 non-null    object 
 6   Alley          9 non-null      object 
 7   LotShape       147 non-null    object 
 8   LandContour    147 non-null    object 
 9   Utilities      147 non-null    object 
 10  LotConfig      147 non-null    object 
 11  LandSlope      147 non-null    object 
 12  Neighborhood   147 non-null    object 
 13  Condition1     147 non-null    object 
 14  Condition2     147 non-null    object 
 15  BldgType       147 non-null    object 
 16  HouseStyle     147 non-null    object 
 17  OverallQual    147 non-null    int64  
 18  OverallC

In [5]:
from pycaret.regression import *
data = setup(data=train, target='SalePrice', train_size=.99, numeric_imputation='median', normalize=True, normalize_method='robust')

models()

#best_specific = compare_models(include = ['dt','rf','xgboost', 'ada', 'lr'])
best_specific = compare_models(include = ['dt','rf','ada', 'lr'])

save_model(best_specific,'final_model')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,18136.3269,908839394.9780,29369.9510,0.8558,0.1514,0.1081,1.9140
ada,AdaBoost Regressor,24889.9628,1299455745.1483,35675.9467,0.7943,0.2051,0.1638,0.5940
lr,Linear Regression,18829.3994,1625184166.1754,36403.5116,0.7170,0.1793,0.1118,0.0540
dt,Decision Tree Regressor,27767.2004,2029352482.1778,44404.7156,0.6763,0.2210,0.1621,0.7840


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=['Id'],
                                       ml_usecase='regression',
                                       numerical_features=[], target='SalePrice',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_st...
                  RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                        criterion='mse', max_depth=None,
                                        max_features='auto', max_leaf_nodes=None,
                                        max_samp